In [1]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [ ]:
ALL_IN_ONE_SG = 'sg-0d8a868137f653df6'
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
AWS_INSTANCE_ID_JMASTER   = os.environ['AWS_INSTANCE_ID_JMASTER']
AWS_INSTANCE_ID_JAGENT    = os.environ['AWS_INSTANCE_ID_JAGENT']
AWS_DEFAULT_IMAGE_ID      = os.environ['AWS_DEFAULT_IMAGE_ID']
AWS_DEFAULT_KEY_PAIR_NAME = os.environ['AWS_DEFAULT_KEY_PAIR_NAME']
AWS_DEFAULT_INSTANCE_TYPE = os.environ['AWS_DEFAULT_INSTANCE_TYPE']

In [ ]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')
sqs_client           = boto3.client('sqs')

emr_client = boto3.client('emr', region_name=REGION)

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)

# # Example: Get a specific VPC
# vpc = ec2_resource.Vpc('vpc_id')

# # Example: Get a specific EBS volume
# volume = ec2_resource.Volume('volume_id')

##   [Auto Scaling and Load Balancing on AWS](https://www.youtube.com/watch?v=0mwgbiJae5Q&list=PLO95rE9ahzRs0QMA8qtIAstWFo4X4gHtH&index=5)

In [ ]:
vpc_cidr_block = '10.0.0.0/16'
vpc_name = 'httx-asg-alb'
vpc_id = ec2_client.create_vpc(CidrBlock=vpc_cidr_block)['Vpc']['VpcId']


In [ ]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a', 'tag': 'public-subnet-01'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b', 'tag': 'public-subnet-02'},
    {'cidr_block': '172.0.3.0/24', 'az': 'us-east-1a', 'tag': 'private-subnet-01'},
    {'cidr_block': '172.0.4.0/24', 'az': 'us-east-1b', 'tag': 'private-subnet-02'}
]

In [ ]:
public_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[0]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[0]['az']
)
ec2_client.create_tags(Resources=[public_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[0]['tag']}])

public_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[1]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[1]['az']
)
ec2_client.create_tags(Resources=[public_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[1]['tag']}])

In [ ]:
private_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[2]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[2]['az']
)
ec2_client.create_tags(Resources=[private_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[2]['tag']}])

private_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[3]['cidr_block'],
    VpcId=vpc.id,
    AvailabilityZone=subnet_configs[3]['az']
)
ec2_client.create_tags(Resources=[private_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[3]['tag']}])

- Create Security Group

In [ ]:
ALB1 = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "SSH_Port"}]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 80,
        'ToPort': 80,
        'IpRanges': [{'CidrIp': '0.0.0.0/0', "Description": "HTTP_Port"}]
    },
]

ALL_IN_ONE_OUTBOUND_RULES = [
    {
        'IpProtocol': '-1', # '-1' means all protocols
        'FromPort': -1,
        'ToPort': -1,
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
    }
]

tags = [
    {
        'Key': 'Name',
        'Value': 'ALB1'
    }
]


In [ ]:
ALB1_SG = ec2.create_security_group(
    "ALB1",
    vpc_id,
    inbound_rules=ALB1,
    outbound_rules=ALL_IN_ONE_OUTBOUND_RULES,
    tags=tags,
    description='ALB1'
)['GroupId']

In [ ]:
WEB1_SG_RULE = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 80,
        'ToPort': 80,
        'UserIdGroupPairs': [
            {
                'GroupId': ALB1_SG,
                'Description': 'Allow all tcp traffic from another security group on port 80'
            }
        ]
    },
]

WEB1_SG = ec2.create_security_group(
    "WEB1",
    vpc_id,
    inbound_rules=WEB1_SG_RULE,
    outbound_rules='',
    tags=[{'Key': 'Name','Value': 'WEB-1'}],
    description='WEB-1'
)['GroupId']

In [ ]:
ec2.allow_sg_to_sg(WEB1_SG, source_security_group_id=ALB1_SG)